# Redo download

I need to redo all of my downloads, because I had been doing it wrong. So this notebook does that.

1. 

2. Download tars based on metadata.
    - If tars exist on 

In [79]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import boto3, configparser, os, pandas as pd, shutil
import datetime, urllib, numpy as np, tarfile, gzip
from bs4 import BeautifulSoup

## Query metadata

In [80]:
def request_bulk_metadata(date_of_last_request):  
    rows = []
    resumptionToken = 'placeholder'
    url = 'http://export.arxiv.org/oai2?verb=ListRecords&set=physics:astro-ph&metadataPrefix=arXiv'
    
    # If we have specified the date of the last request, add it to the URL
    if date_of_last_request:
        url += '&from=' + date_of_last_request.strftime('%Y-%m-%d')

    # Continue requesting until we are not given any more resumption tokens
    while resumptionToken is not None:
        # Send request and receive results
        print('Requesting: ' + url)
        results = urllib.request.urlopen(url).read()

        # Parse with Beautiful Soup
        soup = BeautifulSoup(results, 'xml')
        records = soup.find_all('record')
        for record in records:
            # Get header data
            identifier = record.find('identifier')
            datestamp = record.find('datestamp')
            spec = record.find('setSpec')

            # Get metadata
            filename = record.find('id')
            created = record.find('created')
            updated = record.find('updated')
            authors = []
            for author in record.find_all('author'):
                forenames = author.forenames
                keyname = author.keyname
                if forenames and keyname:
                    authors.append(author.forenames.text.strip() + ' ' + author.keyname.text.strip())
            author_str = ', '.join(authors)
            title = record.find('title')
            categories = record.find('categories')
            journal = record.find('journal-ref')
            doi = record.find('doi')
            abstract = record.find('abstract')
            comments = record.find('comment')

            # Save current record as a row in the table
            row = {
                'identifier': getattr(identifier, 'text', None),
                'filename': getattr(filename, 'text', None),
                'spec': getattr(spec, 'text', None),
                'title': getattr(title, 'text', None),
                'datestamp': getattr(datestamp, 'text', None),
                'created': getattr(created, 'text', None),
                'updated': getattr(updated, 'text', None), # may have more than one instance that we're missing
                'authors': author_str,
                'categories': getattr(categories, 'text', None),
                'journal': getattr(journal, 'text', None),
                'doi': getattr(doi, 'text', None),
                'abstract': getattr(abstract, 'text', None),
                'comments': getattr(comments, 'text', None)
            }
            rows.append(row)

        # Get resumption token if provided
        resumptionToken = soup.find('resumptionToken')

        # Continue if we have resumption token
        if resumptionToken is not None:
            print('Status: ' + str(int(resumptionToken['cursor']) + 1) + '—' + str(len(rows)) + '/' + str(resumptionToken['completeListSize']) + '...')
            resumptionToken = resumptionToken.text
            url = 'http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=' + resumptionToken
            time.sleep(20) # avoid 503 status
        else:
            # Otherwise, obtain date of last request and the while loop ends here
            requestDate = soup.find('responseDate').text
        
    return rows, requestDate

In [81]:
global metadata_filepath
global identifiers
metadata_filepath = 'arxiv_metadata_astroph.csv'

if os.path.exists(metadata_filepath):
    # If the metadata file exists, load it into a data frame
    existing_metadata_df = pd.read_csv(metadata_filepath, 
                                       dtype={'filename': str,
                                              'filename_parsed': str,
                                              'identifier': str,
                                              'updated': str,
                                              'doi': str}, 
                                       parse_dates=['date_retrieved'])
    # Get the date of the last request
    date_of_last_request = existing_metadata_df['date_retrieved'].max()
    print(metadata_filepath + ' last updated on ' + date_of_last_request.strftime('%Y-%m-%d'))
    print('Updating...')
    # Send a request to access metadata since that date
    records, requestDate = request_bulk_metadata(date_of_last_request + datetime.timedelta(days=1))
    # Create data frame for records to specify additional info
    if len(records) > 0:
        print('Number of new records found: ' + str(len(records)))
        records_df = pd.DataFrame(records)
        records_df['date_retrieved'] = np.full(len(records_df), requestDate)
        records_df['filename_parsed'] = existing_metadata_df['filename'].str.replace('/', '')
        # Update metadata file
        metadata_df = pd.concat([existing_metadata_df, records_df], axis=0, sort=True, ignore_index=True)
        metadata_df.to_csv(metadata_filepath, index=False)
        print('Metadata has been updated.')
    else: 
        print('No additional records found. Metadata is up to date.')
else:
    # If the metadata file doesn't exist, request all metadata
    print(metadata_filepath + ' is being created...')
    records, requestDate = request_bulk_metadata(None)
    # Load records into data frame
    metadata_df = pd.DataFrame(records)
    # Add a column to specify additional info
    metadata_df['date_retrieved'] = np.full(len(metadata_df), requestDate)
    metadata_df['filename_parsed'] = metadata_df['filename'].str.replace('/', '')
    # Save it to CSV
    metadata_df.to_csv(metadata_filepath, index=False)
    print('Metadata has been saved.')
    
# Grab identifiers 
metadata_df = pd.read_csv(metadata_filepath, 
                           dtype={'filename': str,
                                  'filename_parsed': str,
                                  'identifier': str,
                                  'updated': str,
                                  'doi': str}, 
                           parse_dates=['date_retrieved'])
identifiers = metadata_df['filename_parsed']
identifiers

arxiv_metadata_astroph.csv last updated on 2019-02-22
Updating...
Requesting: http://export.arxiv.org/oai2?verb=ListRecords&set=physics:astro-ph&metadataPrefix=arXiv&from=2019-02-23
No additional records found. Metadata is up to date.


0         0704.0009
1         0704.0017
2         0704.0023
3         0704.0044
4         0704.0048
5         0704.0059
6         0704.0080
7         0704.0094
8         0704.0128
9         0704.0133
10        0704.0138
11        0704.0139
12        0704.0144
13        0704.0155
14        0704.0156
15        0704.0160
16        0704.0168
17        0704.0171
18        0704.0175
19        0704.0184
20        0704.0187
21        0704.0192
22        0704.0203
23        0704.0205
24        0704.0207
25        0704.0209
26        0704.0212
27        0704.0219
28        0704.0221
29        0704.0222
            ...    
251331    0704.0356
251332    0704.0360
251333    0704.0381
251334    0704.0387
251335    0704.0401
251336    0704.0421
251337    0704.0423
251338    0704.0426
251339    0704.0428
251340    0704.0431
251341    0704.0441
251342    0704.0448
251343    0704.0452
251344    0704.0454
251345    0704.0455
251346    0704.0458
251347    0704.0460
251348    0704.0462
251349    0704.0463


## Connect to Google Drive 

Connect to Google Drive to see which tars we have stored ourselves (to save $):

In [3]:
# Set up Google Drive according to docs
def connect_to_google_drive():
    g_login = GoogleAuth()
    g_login.LocalWebserverAuth()
    drive = GoogleDrive(g_login)
    return drive

drive = connect_to_google_drive()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=205689913441-4qvumj04tvu7o2h0j1cth62qhp0ck9ld.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [18]:
def download_gdrive_file(file, title):
    '''
    Downloads given file from Google Drive.

    Parameters
    ----------
    file : str
        The file in the form of a GoogleFile object 
    title : str
        The filename of the file
    '''
        
    # Ensure src directory exists 
    if not os.path.isdir('src'):
        os.makedirs('src')
    
    # Download file
    print('Downloading ' + title + '...')
    file.GetContentFile(title) 
    print('Successfully downloaded drive://arxiv/{} to {}'.format(os.path.basename(title), title))
    

def query_tarfiles_from_arxiv_folder_on_gdrive(key):
    '''
    Gets names of all tarfiles in arxiv folder on Google Drive. 
    Returns list of strings, each string representing a name. 
    If there is no folder or it is empty, an empty list is returned. 
    '''
    
    gdrive_tars = []
    query = "'root' in parents and trashed=false and title='arxiv' and mimeType='application/vnd.google-apps.folder'"
    arxiv_folder_id = drive.ListFile({'q': query}).GetList()[0].metadata['id']
    
    # If folder doesn't exist, create and upload it, exit (we will have to query everything from S3)
    if not arxiv_folder_id:
        print('Google Drive does not contain a folder named \'arxiv\'. Creating folder...')
        arxiv_folder = drive.CreateFile({'title': 'arxiv', 
                                    "mimeType": "application/vnd.google-apps.folder"}) 
        arxiv_folder.Upload()
        arxiv_folder_id = arxiv_folder['id']
        print('Folder created.')
    else:
        # Get list of files in arxiv folder
        gdrive_tars = drive.ListFile({'q': "'" + arxiv_folder_id + "' in parents and trashed=false"}).GetList()
        gdrive_tars_str = [x.metadata['title'] for x in gdrive_tars]
        print('Number of tars on Google Drive: ' + str(len(gdrive_tars)))
        gdrive_tars_str.sort()
    
    return gdrive_tars

In [43]:
gdrive_tars = query_tarfiles_from_arxiv_folder_on_gdrive()

Number of tars on Google Drive: 1388


## Conversion

In [ ]:
def identify_preprints(folder):
    '''
    Collects the filepaths for all preprints within 
    the given folder. Returns an array of strings,
    each string representing the path of a preprint.
    '''
    
    # Initialize variables
    global empty_submissions 
    empty_submissions = []
    global preprints 
    preprints = []
    global corrupt_submissions 
    corrupt_submissions = []
    base_path = folder
    submission_count = 0
    texfile_count = 0

    # Walk through tar directories
    for idx, tar_folder in enumerate(os.listdir(base_path)):
        
        # If current path isn't a directory, skip
        tar_path = base_path + '/' + tar_folder
        if not os.path.isdir(tar_path):
            continue
        
        # Walk through each submission directory
        submission_dirs = os.listdir(tar_path)
        submission_count += len(submission_dirs)
        for submission in submission_dirs:
            
            # If current path isn't a directory, skip
            submission_path = tar_path + '/' + submission
            if not os.path.isdir(submission_path):
                submission_count -= 1
                continue

            arxiv_id = os.path.basename(submission_path) # used to note empty or corrupt submissions 

            # If submission is empty, note & skip
            texs = glob.glob(submission_path + '/**/*.tex', recursive=True)
            texfile_count += len(texs)
            if len(texs) == 0:
                empty_submissions.append(arxiv_id)
                continue
            
            # Otherwise get the preprint
            else:
                preprint_path = identify_preprint(submission_path, texs)
                if preprint_path:
                    preprints.append(preprint_path)
                else:
                    corrupt_submissions.append(arxiv_id)
    
    print('TEX files: ' + str(texfile_count))
    print('Submissions: ' + str(submission_count))
    print('Preprints: ' + str(len(preprints)))
    print('Empty submissions: ' + str(len(empty_submissions)))
    print('Potentially corrupt submissions: ' + str(len(corrupt_submissions)))

## Connect to Amazon S3

In [79]:
global s3resource 

def setup():
    '''
    Creates S3 resource & sets configs to enable download.
    '''

    # Securely import configs from private AWS config file
    configs = configparser.ConfigParser()
    configs.read('config.ini')

    # Create S3 resource & set configs
    global s3resource
    s3resource = boto3.resource(
        's3',  # the AWS resource we want to use
        aws_access_key_id=configs['DEFAULT']['ACCESS_KEY'],
        aws_secret_access_key=configs['DEFAULT']['SECRET_KEY'],
        region_name='us-east-1'  # same region the arxiv bucket is in
    )

def start():
    '''
    Queries S3 arxiv bucket to see which tar files to download,
    and calls download_file() on the appropriate tar files. 
    '''
    
    print('Querying S3 arxiv bucket...')
    setup()

    # Create a reusable Paginator
    paginator = s3resource.meta.client.get_paginator('list_objects_v2')

    # Create a PageIterator from the Paginator
    page_iterator = paginator.paginate(
        Bucket='arxiv',
        RequestPayer='requester',
        Prefix='src/'
    )

    # Download and extract tars
    numFiles = 0
    gdrive_tar = None
    for page in page_iterator:
        numFiles = numFiles + len(page['Contents'])
        for file in page['Contents']:
            key = file['Key']
            # If current file is a tar 
            if key.endswith('.tar'):
                # Check if file exists in Google Drive 
                for google_file in gdrive_tars:
                    if google_file.metadata['title'] == os.path.basename(key):
                        gdrive_tar = google_file
                        break
                # If it is in Google Drive, download it, extract, convert, delete
                if gdrive_tar:
                    download_gdrive_file(gdrive_tar, key)
                    extract(key)
                    # Convert... [needed]
                    os.remove(key)
                # Otherwise download it from S3
                else:
                    download_file(key)
                    extract(key)
                    # Convert... [needed]
                    # Upload .tar to Google Drive for storage
                    os.remove(key)
            
    print('Processed ' + str(numFiles - 1) + ' tars')  # -1 
    
def download_file(key):
    '''
    Downloads given filename from source bucket to destination directory.

    Parameters
    ----------
    key : str
        Name of file to download
    '''

    # Ensure src directory exists 
    if not os.path.isdir('src'):
        os.makedirs('src')
    
    print('Downloading s3://arxiv/{}'.format(key))
    
    # Download file
    try:
        s3resource.meta.client.download_file(
            Bucket='arxiv', 
            Key=key,  # name of key to download from
            Filename=key,  # path to file to download to
            ExtraArgs={'RequestPayer':'requester'})
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print('ERROR: ' + key + " does not exist in arxiv bucket")
            
    print('Successfully downloaded s3://arxiv/{} to {}'.format(key, key))

Querying S3 arxiv bucket...
Successfully downloaded drive://arxiv/arXiv_src_0001_001.tar to src/arXiv_src_0001_001.tar
Opening src/arXiv_src_0001_001.tar


NameError: name 'gzip' is not defined

I'll need to log which astrophysics articles are in which tars. 

## General functions

In [108]:
def extract(filepath):
    '''
    Extracts astro-ph submissions from given tar filepath.
    '''
    
    global identifiers 
    
    # Quit file extraction if given file is not tarfile
    if not tarfile.is_tarfile(filepath):
        print('can\'t unzip {}, not a .tar file'.format(filepath))
    
    total_tex = 0
    tar_dir = 'latex/' + os.path.splitext(os.path.basename(filepath))[0] + '/'

    # Create .tar directory if it doesn't exist
    if not os.path.isdir(tar_dir):
        os.makedirs(tar_dir)
        
    # Open tarfile, read-only
    print('Opening {}'.format(filepath))
    tar = tarfile.open(filepath)
    # Iterate over submissions, extracting only those that belong to the astro-ph category
    for submission in tar.getmembers():
        # Create submission directory if it doesn't exist
        submission_id = os.path.splitext(os.path.basename(submission.name))[0]
        submission_path = tar_dir + '/' + submission_id
        if submission.name.endswith('.gz') and identifiers.str.contains(submission_id).any():
            if not os.path.isdir(submission_path):
                os.makedirs(submission_path)
            #### Extract here.........
            try:
                gz_obj = tar.extractfile(submission) 
                gz = tarfile.open(fileobj=gz_obj) 
                gz.extractall(path=submission_path)
                total_tex += 1
            except tarfile.ReadError:
                # Extract the entire .gz because we cannot read it using tarfile 
                # Note that these .gzs are single .tex files with no extension specified
                tar.extract(submission, path='temp')
                # Uncompress the .gz file using gzip instead and place it with the other .tex files
                with gzip.open('temp/' + submission.name, 'rb') as f_in:
                    basename = os.path.splitext(os.path.basename(submission.name))[0]
                    with open(tar_dir + submission_id + '/' + basename + '.tex', 'wb+') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                        total_tex += 1
    tar.close()
    
    print(filepath + ' extraction complete')
    print('Number of .tex files obtained: ' + str(total_tex) + '\n')

In [109]:
start()

Querying S3 arxiv bucket...
Successfully downloaded drive://arxiv/arXiv_src_0001_001.tar to src/arXiv_src_0001_001.tar
Opening src/arXiv_src_0001_001.tar
src/arXiv_src_0001_001.tar extraction complete
Number of .tex files obtained: 600



OSError: [Errno 49] Can't assign requested address

In [116]:
setup()

# Create a reusable Paginator
paginator = s3resource.meta.client.get_paginator('list_objects_v2')

# Create a PageIterator from the Paginator
page_iterator = paginator.paginate(
    Bucket='arxiv',
    RequestPayer='requester',
    Prefix='src/'
)

for page in page_iterator:
    files = [file['Key'] for file in page['Contents'] if file['Key'].endswith('.tar') and not os.path.isdir(''.join(['latex/', os.path.splitext(os.path.basename(file['Key']))[0]]))]
    break
             
files

['src/arXiv_src_0002_001.tar',
 'src/arXiv_src_0003_001.tar',
 'src/arXiv_src_0004_001.tar',
 'src/arXiv_src_0005_001.tar',
 'src/arXiv_src_0006_001.tar',
 'src/arXiv_src_0007_001.tar',
 'src/arXiv_src_0008_001.tar',
 'src/arXiv_src_0009_001.tar',
 'src/arXiv_src_0010_001.tar',
 'src/arXiv_src_0011_001.tar',
 'src/arXiv_src_0012_001.tar',
 'src/arXiv_src_0101_001.tar',
 'src/arXiv_src_0102_001.tar',
 'src/arXiv_src_0103_001.tar',
 'src/arXiv_src_0104_001.tar',
 'src/arXiv_src_0105_001.tar',
 'src/arXiv_src_0106_001.tar',
 'src/arXiv_src_0107_001.tar',
 'src/arXiv_src_0108_001.tar',
 'src/arXiv_src_0109_001.tar',
 'src/arXiv_src_0110_001.tar',
 'src/arXiv_src_0111_001.tar',
 'src/arXiv_src_0112_001.tar',
 'src/arXiv_src_0201_001.tar',
 'src/arXiv_src_0202_001.tar',
 'src/arXiv_src_0203_001.tar',
 'src/arXiv_src_0204_001.tar',
 'src/arXiv_src_0205_001.tar',
 'src/arXiv_src_0206_001.tar',
 'src/arXiv_src_0207_001.tar',
 'src/arXiv_src_0208_001.tar',
 'src/arXiv_src_0209_001.tar',
 'src/ar

In [2]:
import os
''.join(['latex/', os.path.splitext(os.path.basename('src/arXiv_src_0003_001.tar'))[0]])

'latex/arXiv_src_0003_001'

In [15]:
import os, re, shutil

In [16]:
with open ('logs/tarfile_submission_log.txt', 'r') as logfile:
    tar_dir = None
    converts = os.listdir('xml')
    
    for line in logfile:
        regex = re.compile(r'TARFILE: (.*)\.')
        matches = regex.match(line)
        if matches:
            tar_dir = 'latex/' + matches.group(1)
            # Check that all files have been converted, if not, ensure it is removed from /latex
        elif line != '\n' and not os.path.isdir(tar_dir):
            # Skip until the end of submission list, prevents os from doing unnecessary checking
            pass
        elif line != '\n':
            converted_id = 'xml/' + line.strip() + '.xml'
            if converted_id not in converts:
                print('Needs to be restarted: {}'.format(tar_dir))
                shutil.rmtree(tar_dir, ignore_errors=True)
                

Needs to be restarted: latex/arXiv_src_0411_002
Needs to be restarted: latex/arXiv_src_0902_001
Needs to be restarted: latex/arXiv_src_1011_007
Needs to be restarted: latex/arXiv_src_1203_002
Needs to be restarted: latex/arXiv_src_1012_001
Needs to be restarted: latex/arXiv_src_1302_008
Needs to be restarted: latex/arXiv_src_1312_001
Needs to be restarted: latex/arXiv_src_1302_009
Needs to be restarted: latex/arXiv_src_1404_012
Needs to be restarted: latex/arXiv_src_1409_005
Needs to be restarted: latex/arXiv_src_1312_002
Needs to be restarted: latex/arXiv_src_1404_013
Needs to be restarted: latex/arXiv_src_1302_010
Needs to be restarted: latex/arXiv_src_1405_001
Needs to be restarted: latex/arXiv_src_1312_003
Needs to be restarted: latex/arXiv_src_1501_003
Needs to be restarted: latex/arXiv_src_1504_017
Needs to be restarted: latex/arXiv_src_1505_006
Needs to be restarted: latex/arXiv_src_1303_006


In [17]:
# Check for erroneous conversions at the end of the conversion process.
# Check if a new .xml file has been created. If not, add a .txt.
# This .txt marks that it failed. Also add .txt if TimeoutExpired exception is thrown. 

In [19]:
import re

In [61]:
candidates = 

with open('/Users/briennakh/Downloads/1003.2415/template-6s.tex', 'rb') as f:
    data = f.read()
    doc_regex = re.compile(b'(?m)^\\\\document(?:class|style).*')
    if not doc_regex.findall(data):
        print('remove from candidates')

remove from candidates


Convert .gz to .zip

In [76]:
import tarfile, zipfile, time
start = time.time()
tarf = tarfile.open('/Users/briennakh/Downloads/astro-ph0008018.gz', mode='r|gz')
zipf = zipfile.ZipFile(file='myzip.zip', mode='a', compression=zipfile.ZIP_DEFLATED)
for m in tarf:
    f = tarf.extractfile(m)
    f1 = f.read()
    fn = m.name
    zipf.writestr(fn, f1)
tarf.close()
zipf.close()

print('took ' + str(time.time() - start) + 's')

ReadError: invalid header

In [142]:
def extract(filepath, identifiers):
    if not tarfile.is_tarfile(filepath):
        print('can\'t unzip {}, not a .tar file'.format(filepath))
        return
    
    total_submissions_extracted = 0
    tar_dir = 'latex/' + os.path.splitext(os.path.basename(filepath))[0] 
    # confirmDir(tar_dir)
    if not os.path.isdir(tar_dir):
        os.makedirs(tar_dir)
    
    tar = tarfile.open(filepath)
    for submission in tar.getmembers():
        submission_id = os.path.splitext(os.path.basename(submission.name))[0]
        if submission.name.endswith('.gz') and identifiers.str.contains(submission_id).any():
            submission_path = tar_dir + '/' + submission_id
            try:    
                # Extract and convert to .zip
                gz_obj = tar.extractfile(submission)
                gz = tarfile.open(fileobj=gz_obj)
                zipf = zipfile.ZipFile(file=submission_path + '.zip', mode='a', compression=zipfile.ZIP_DEFLATED)                
                            
                for m in gz: 
                    f = gz.extractfile(m)
                    if m.isdir():
                        print(submission_id)
                        continue
                    f_out = f.read()
                    f_in = m.name
                    zipf.writestr(f_in, f_out)
                zipf.close()
                gz.close()
                total_submissions_extracted += 1
            except tarfile.ReadError:
                if not os.path.isdir('temp'):
                    os.makedirs('temp')
                tar.extract(submission, 'temp')
                with gzip.open('temp/' + submission.name, 'rb') as f_in:
                    with open(submission_path + '.tex', 'wb+') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                        total_submissions_extracted += 1
                        
    tar.close()
    shutil.rmtree('temp/', ignore_errors=True)
    print(filepath + ' extraciton completed')
    print('Number of submissions obtained: ' + str(total_submissions_extracted))

    
extract('src/arXiv_src_0008_001.tar', identifiers)
    

astro-ph0008115


/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figs/annih_WW.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figs/annih_ZZ.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figs/annih_ff.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figs/captureE.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figs/captureS.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figs/dFdEPos.eps'
  return self._open_to_write(zinfo, force_zip64=force_

/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'bhind14.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'bhind2.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'bhind3.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'bhind4.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'bhind5.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'bhind6,ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.

/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'h2211f7.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'h2211f8.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'h2211f9.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'Arp119.tex'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figure1a.gif'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'figure1b.gif'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/

/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'A2218Lens.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'AmpMap.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'BU_intersection.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'contrib.tex'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'ncrit.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'HD100546_adonis_2.tex'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/brienna

...

/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'aat00a_f1BW.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'abABW.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'ryanwill2.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'torino00.tex'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: '0614nur.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: '1728cc.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/

/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'H2415f4d.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'H2415f4e.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'H2415f5a.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'H2415f5b.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'H2415f6a.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'H2415f6b.eps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaco

/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'ms14.tex'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'fig1a.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'fig1b.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'fig2.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'fig3.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'fig4.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zip

/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'paper26.tex'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'persistent.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'plat96.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'rel_corr.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'thins.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'transient.ps'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/briennakh/anaconda3/lib

astro-ph0008468
astro-ph0008468
src/arXiv_src_0008_001.tar extraciton completed
Number of submissions obtained: 564


In [ ]:
def write_to_zip(directory, zipf, tar):
    '''
    Loops over given directory recursively until all files have been read
    directory is a gz file that has been opened by tarfile.
    '''
    
    for item in directory: 
        if item.isdir():
            print('Going into ' + str(item))
            item_obj = tar.extractfile(item)
            print(type(item_obj))
            write_to_zip(tarfile.open(fileobj=item_obj), zipf)
        else:
            f = directory.extractfile(item)
            f_out = f.read()
            f_in = item.name
            zipf.writestr(f_in, f_out)
            
def extract(filepath, identifiers):
    tar_dir = 'latex/' + os.path.splitext(os.path.basename(filepath))[0] 
    tar = tarfile.open(filepath)
    for submission in tar.getmembers():
        submission_id = os.path.splitext(os.path.basename(submission.name))[0]
        if submission.name.endswith('.gz') and identifiers.str.contains(submission_id).any():
            submission_path = tar_dir + '/' + submission_id
            gz_obj = tar.extractfile(submission)
            gz = tarfile.open(fileobj=gz_obj)
            zipf = zipfile.ZipFile(file=submission_path + '.zip', mode='a', compression=zipfile.ZIP_DEFLATED)                
            write_to_zip(gz, zipf, tar)
            zipf.close()
            gz.close()

In [143]:
import glob

In [146]:
tar = 'latex/arXiv_src_0008_001'

glob.glob(tar + '/*[.tex|.zip]')

['latex/arXiv_src_0008_001/astro-ph0008001.zip',
 'latex/arXiv_src_0008_001/astro-ph0008002.zip',
 'latex/arXiv_src_0008_001/astro-ph0008003.zip',
 'latex/arXiv_src_0008_001/astro-ph0008004.zip',
 'latex/arXiv_src_0008_001/astro-ph0008005.zip',
 'latex/arXiv_src_0008_001/astro-ph0008006.zip',
 'latex/arXiv_src_0008_001/astro-ph0008007.zip',
 'latex/arXiv_src_0008_001/astro-ph0008008.zip',
 'latex/arXiv_src_0008_001/astro-ph0008009.zip',
 'latex/arXiv_src_0008_001/astro-ph0008010.zip',
 'latex/arXiv_src_0008_001/astro-ph0008011.zip',
 'latex/arXiv_src_0008_001/astro-ph0008012.zip',
 'latex/arXiv_src_0008_001/astro-ph0008013.zip',
 'latex/arXiv_src_0008_001/astro-ph0008014.zip',
 'latex/arXiv_src_0008_001/astro-ph0008015.zip',
 'latex/arXiv_src_0008_001/astro-ph0008016.zip',
 'latex/arXiv_src_0008_001/astro-ph0008017.zip',
 'latex/arXiv_src_0008_001/astro-ph0008018.tex',
 'latex/arXiv_src_0008_001/astro-ph0008019.zip',
 'latex/arXiv_src_0008_001/astro-ph0008020.zip',
 'latex/arXiv_src_00

In [148]:
def confirmDir(dir_name):
	if not os.path.isdir(dir_name):
		os.makedirs(dir_name)

def extract(filepath, identifiers):
	'''
	Extracts astro-ph submissions from given tar filepath.
	Logs which submissions belong to particular tarfile.
	'''

	# Quit if given file is not tarfile
	if not tarfile.is_tarfile(filepath):
		print('can\'t unzip {}, not a .tar file'.format(filepath))
		return

	total_submissions_extracted = 0
	tar_dir = 'latex/' + os.path.splitext(os.path.basename(filepath))[0] + '/'
	confirmDir(tar_dir)
		
	# Open tarfile, read-only
	print('Extracting {}'.format(filepath))
	tar = tarfile.open(filepath)
	# Iterate over submissions, extracting only those that belong to the astro-ph category, 
	# logging which submissions belong to which tarfile
	with open(tar_dir + 'submissions.txt', 'w+') as logfile:
		logfile.write('TARFILE: {}'.format(os.path.basename(filepath)))
		for submission in tar.getmembers():
			submission_id = os.path.splitext(os.path.basename(submission.name))[0]
			if submission.name.endswith('.gz') and identifiers.str.contains(submission_id).any():
				logfile.write('\n' + submission_id)
				submission_path = tar_dir + submission_id
				# Extract the submission as a .gzip
				try:    
					# Extract and convert to .zip
					gz_obj = tar.extractfile(submission)
					gz = tarfile.open(fileobj=gz_obj)
					zipf = zipfile.ZipFile(file=submission_path + '.zip', mode='a', compression=zipfile.ZIP_DEFLATED)                
								
					for m in gz: 
						f = gz.extractfile(m)
						if m.isdir():
							continue
						f_out = f.read()
						f_in = m.name
						zipf.writestr(f_in, f_out)
					zipf.close()
					gz.close()
					total_submissions_extracted += 1
				except tarfile.ReadError:
					confirmDir('temp')
					tar.extract(submission, 'temp')
					with gzip.open('temp/' + submission.name, 'rb') as f_in:
						with open(submission_path + '.tex', 'wb+') as f_out:
							shutil.copyfileobj(f_in, f_out)
							total_submissions_extracted += 1
	tar.close()
	# Delete the temporary folder for those wonky gz files
	shutil.rmtree('temp/', ignore_errors=True)
	print(filepath + ' extraction complete')
	print('Number of submissions obtained: ' + str(total_submissions_extracted))

extract('src/arXiv_src_1003_006.tar', identifiers)

Extracting src/arXiv_src_1003_006.tar
1003.5774
1003.5774
1003.5846
1003.5912
1003.6031
src/arXiv_src_1003_006.tar extraction complete
Number of submissions obtained: 100
